In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import (
    LogisticRegression, PassiveAggressiveClassifier,
    Perceptron, RidgeClassifier, SGDClassifier,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (BaggingClassifier,
                              GradientBoostingClassifier,
                              RandomForestClassifier)
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score,
    mean_squared_error,
    f1_score,
    mean_absolute_error,
    confusion_matrix
)

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
!pip install joblib

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pretty_midi
import collections
from typing import Dict, List, Optional, Sequence, Tuple
import math
import string
import gensim
import ast
import sentencepiece as spm
from joblib import dump

In [ ]:
music_path_parent = "/content/drive/MyDrive/dataset/maestro-v3.0.0/"

In [ ]:
def remove_extra(use):
    new_names = np.array([])
    for i in range(use.shape[0]):
        new_name = use["canonical_composer"][i].split("/")[0].strip()
        new_names = np.append(new_names,new_name)

    ser_names = pd.Series(new_names)
    use["canonical_composer"] = ser_names
    return use

In [ ]:
music_data = pd.read_csv("/content/drive/MyDrive/Music Machine Learning Project/maestro-v3.0.0.csv")
music_data = remove_extra(music_data)
music_data['canonical_composer'].nunique()

43

In [ ]:
composers_filter = music_data[music_data["canonical_composer"].isin([
    'Frédéric Chopin', 'Franz Schubert', 'Ludwig van Beethoven',
       'Johann Sebastian Bach', 'Franz Liszt', 'Sergei Rachmaninoff',
       'Robert Schumann', 'Claude Debussy', 'Joseph Haydn',
       'Wolfgang Amadeus Mozart'])].reset_index().drop(columns=['index'], axis=['columns'])

In [ ]:
composers_filter['canonical_composer'].value_counts()

Frédéric Chopin            201
Franz Schubert             197
Johann Sebastian Bach      156
Ludwig van Beethoven       146
Franz Liszt                134
Sergei Rachmaninoff         61
Robert Schumann             50
Claude Debussy              45
Joseph Haydn                40
Wolfgang Amadeus Mozart     38
Name: canonical_composer, dtype: int64

In [ ]:
# 1
def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    notes['velocity'].append(note.velocity)
    prev_start = start
  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [ ]:
# 2
def extract_gram(midi_frame):
  gram_list = []
  temp = []
  s_time = 0
  for i in range(midi_frame.shape[0]):
    pitch = midi_frame["pitch"][i]
    ti = round(midi_frame["duration"][i],2)
    gram = (pitch,ti)

    if((not temp) or (midi_frame["start"][i] - s_time <= 0.003)):
      temp.append(gram)
      if(len(temp) == 1):
        s_time = midi_frame["start"][i]
      if(i == midi_frame.shape[0] - 1):
        gram_list += temp
    else:
      sorted_list = sorted(temp, key=lambda tup: tup[0], reverse=True)
      sorted_list.append(gram)
      gram_list += sorted_list
      temp.clear()
      s_time = 0

  return gram_list

In [ ]:
def encodeChinese(index_number):
  val = index_number + 0x4e00
  return chr(val)

In [ ]:
def get_sentence_vec_avg(sentences,model):
  l = []
  for sentence in sentences:
    for word in sentence:
      try:
        temp = np.zeros(len(model.wv[word]))
        temp += model.wv[word]
      except:
        print("Not in vocab")
    l.append(temp/len(sentence))
  return l

In [ ]:
def get_sentence_vec_avg_with_cov2(sentences,model):
  l = []
  cov = []
  for sentence in sentences:
    for word in sentence:
      try:
        temp = np.zeros(len(model.wv[word]))
        temp += model.wv[word]
        cov.append(model.wv[word])
      except:
        print("Not in vocab")
    data = np.array(cov)
    sd = np.std(data,axis=0)
    z = temp/len(sentence)
    z = z.tolist()
    z += sd.tolist()
    z = np.array(z)
    l.append(z)
  return l

In [ ]:
def get_sentence_vec_SD_only(sentences,model):
  l = []
  cov = []
  for sentence in sentences:
    for word in sentence:
      try:
        cov.append(model.wv[word])
      except:
        print("Not in vocab")
    data = np.array(cov)
    sd = np.std(data,axis=0)
    z = sd.tolist()
    z = np.array(z)
    l.append(z)
  return l

In [ ]:
composers_filter.head()

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Claude Debussy,"""Feux d'artifice"" from Book II",train,2008,2008/MIDI-Unprocessed_10_R3_2008_01-05_ORIG_MI...,2008/MIDI-Unprocessed_10_R3_2008_01-05_ORIG_MI...,242.752161
1,Claude Debussy,"""Images"", Series I",train,2013,2013/ORIG-MIDI_03_7_10_13_Group_MID--AUDIO_18_...,2013/ORIG-MIDI_03_7_10_13_Group_MID--AUDIO_18_...,918.021251
2,Claude Debussy,"""Les collines d'Anacapri"" from Preludes, Book I",train,2008,2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...,2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...,166.495560
3,Claude Debussy,"""Ondine"" from Book II",train,2008,2008/MIDI-Unprocessed_10_R3_2008_01-05_ORIG_MI...,2008/MIDI-Unprocessed_10_R3_2008_01-05_ORIG_MI...,193.301089
4,Claude Debussy,"""Voiles"" from Preludes, Book I",train,2008,2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...,2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...,212.295461


In [ ]:
def createLabel(sentenceslst):
  composer_music = composers_filter.groupby(["canonical_composer"])["canonical_title"].count().to_frame()
  composer_music.columns = ["canonical_title"]
  composer_music.reset_index(inplace=True)
  composer_list= []
  for i in range(composer_music.shape[0]):
    composer_list.append(composer_music.iloc[i][0])
  composer_map = { j:i for i,j in enumerate(composer_list)}
  data = []
  label = []
  for i in composers_filter.index.tolist():
    try:
      data.append(sentenceslst[i])
      label.append(composer_map[composers_filter.iloc[i]["canonical_composer"]])
    except:
      print("Error",i)
  return data, label

In [ ]:
all_note_list = []
for i in range(composers_filter.shape[0]):
  if i % 100 == 0:
    print(i)
  suffix_path = composers_filter["midi_filename"][i]
  path = music_path_parent + suffix_path
  frame = midi_to_notes(path)
  gram_list = extract_gram(frame)
  all_note_list = all_note_list + gram_list

0


KeyboardInterrupt: ignored

In [ ]:
np.save("/content/drive/MyDrive/Composer-Classification/all_gram_list_note_as_char2.npy", all_note_list)

In [ ]:
all_music_list = np.load("/content/drive/MyDrive/Composer-Classification/all_gram_list_note_as_char2.npy")

In [ ]:
all_music_list[:20]

array([[6.5e+01, 6.0e-02],
       [6.7e+01, 5.0e-02],
       [6.9e+01, 5.0e-02],
       [7.0e+01, 5.0e-02],
       [6.8e+01, 6.0e-02],
       [6.6e+01, 5.0e-02],
       [6.5e+01, 5.0e-02],
       [6.7e+01, 4.0e-02],
       [6.9e+01, 5.0e-02],
       [7.0e+01, 6.0e-02],
       [6.8e+01, 6.0e-02],
       [6.6e+01, 5.0e-02],
       [6.5e+01, 7.0e-02],
       [6.7e+01, 5.0e-02],
       [6.9e+01, 5.0e-02],
       [7.0e+01, 7.0e-02],
       [6.8e+01, 5.0e-02],
       [6.6e+01, 4.0e-02],
       [6.5e+01, 4.0e-02],
       [6.7e+01, 5.0e-02]])

In [ ]:
sorted_gram_list = sorted(set(tuple(i) for i in all_music_list.tolist()))
note2Ch = { j:encodeChinese(i) for i,j in enumerate(sorted_gram_list)}
Ch2note =  { encodeChinese(i):j for i,j in enumerate(sorted_gram_list)}

In [ ]:
def extractInfoToTxt(filename):
  text = ''
  for i in range(composers_filter.shape[0]):
    if i % 100 == 0:
      print(i)
    suffix_path = composers_filter["midi_filename"][i]
    path = music_path_parent + suffix_path
    frame = midi_to_notes(path)
    gram_list = extract_gram(frame)
    for j in gram_list:
      text += note2Ch[j]
    text += '\n'
  f = open(filename, "w")
  f.write(text)
  f.close()

In [ ]:
extractInfoToTxt("/content/drive/MyDrive/Composer-Classification/CorpusNoteAsChar2.txt")

0
100
200
300
400
500
600
700
800
900
1000


In [ ]:
def sentencePiece(corpus, modelName, vocabSize, maxSenLength):
  spm.SentencePieceTrainer.train(input=corpus, model_prefix=modelName, vocab_size=vocabSize, max_sentence_length=maxSenLength)
  sp = spm.SentencePieceProcessor()
  temp = modelName+".model"
  sp.load(temp)
  f1 = open(corpus,'r')
  temp = {}
  for i in range(1068):
    line = f1.readline()
    tokenized = sp.encode_as_pieces(line)
    temp[i] = tokenized
  Ch_note_series = pd.Series(temp)
  f1.close()
  sentences = []
  for i in range(Ch_note_series.shape[0]):
    sentences.append(Ch_note_series[i])
  return sentences

In [ ]:
def Word2Vec(Window, sentences, Avg = False, SD = False):
  model = gensim.models.Word2Vec(
    sentences=sentences,
    window=Window,
    min_count=1,
    workers=4,
    sg = 1
  )
  if(Avg and SD):
      sentenceLstAvgwithCov = get_sentence_vec_avg_with_cov2(sentences,model)
      return sentenceLstAvgwithCov
  elif(Avg and (not SD)):
      sentencesLstAvg = get_sentence_vec_avg(sentences,model)
      return sentencesLstAvg
  elif((not Avg) and SD):
      sentencesLstSD = get_sentence_vec_SD_only(sentences,model)
      return sentencesLstSD

In [ ]:
def CustomOneHotEncoding(X_train):
    X_train = pd.DataFrame(X_train)
    categorical_columns = X_train.select_dtypes(include=['object']).columns
    df_encoded = pd.get_dummies(X_train, columns=categorical_columns, dtype=np.int16, drop_first=True)
    X_train = pd.concat([X_train, df_encoded], axis=1)
    X_train = X_train.drop(categorical_columns.tolist(), axis='columns')
    return X_train


In [ ]:
class CustomFulledConnectedModel(nn.Module):
    def __init__(self, X_data):
        array_shape = len(X_data[0])
        super(CustomFulledConnectedModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(array_shape),
            nn.Linear(array_shape, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

class RNNModel(nn.Module):
    def __init__(self, input_size):
        super(RNNModel, self).__init__()

        self.rnn = nn.Sequential(
            nn.RNN(input_size, 128, num_layers=1, batch_first=True, dropout=0.0),
            nn.RNN(128, 64, num_layers=1, batch_first=True, dropout=0.0)
        )

        self.fc = nn.Sequential(
            nn.Linear(64, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        out, _ = self.rnn(torch.tensor(x, dtype=torch.float32))
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
def training(corpus, modelName, vocabSize, maxSenLength, window, Avg=False, cov=False):
    sentences = sentencePiece(corpus, modelName, vocabSize, maxSenLength)
    sentencesLst = Word2Vec(window, sentences, Avg, cov)
    data, label = createLabel(sentencesLst)
    sentence_w_label_100 = pd.DataFrame({"sentence": data, "label":label})

    # Create training data
    X = []
    y = []

    for i in composers_filter.index.to_list():
        X.append(sentencesLst[i])
    for i in sentence_w_label_100["label"].tolist():
        y.append(i)

    PredictorScaler=StandardScaler()
    PredictorScalerFit=PredictorScaler.fit(X)
    X=PredictorScalerFit.transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model Training
    models = [LogisticRegression(class_weight='balanced', max_iter=200), PassiveAggressiveClassifier(),
            Perceptron(), RidgeClassifier(max_iter=150), SGDClassifier(), DecisionTreeClassifier(),
            BaggingClassifier(), GradientBoostingClassifier(learning_rate=0.01), RandomForestClassifier(max_depth=500),
            LinearSVC(max_iter=20), KNeighborsClassifier(n_neighbors=10)]

    names = ['LogisticRegression', 'PassiveAggressiveClassifier',
            'Perceptron', 'RidgeClassifier', 'SGD Classifier', 'DecisionTree',
            'BaggingClassifier', 'GradientBoosting', 'RandomForest',
            'SVC', 'KNN', 'Fully-connected', 'RNN']

    evalutions = pd.DataFrame(columns=['score', 'f1-score', 'MSE', 'MAE'], index=names)

    for idx, model in enumerate(models):
        print(f"Running Model {names[idx]}")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        dump(model, names[idx]+'.joblib')
        score = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        model_name = names[idx]
        evalutions.loc[model_name, :] = [score, f1, mse ,mae]

    # Neural Networking Training
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_train = X_train.to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    X_test = X_test.to(device)

    # Number of classes
    num_classes = 10

    # Convert y_train and y_test to PyTorch tensors if they aren't already
    y_train = torch.tensor(y_train, dtype=torch.int64)
    y_test = torch.tensor(y_test, dtype=torch.int64)

    # Convert to one-hot encoding
    y_train_onehot = torch.zeros(y_train.size(0), num_classes)
    y_train_onehot.scatter_(1, y_train.view(-1, 1), 1)
    y_train_onehot = y_train_onehot.to(device)
    y_test_onehot = torch.zeros(y_test.size(0), num_classes)
    y_test_onehot.scatter_(1, y_test.view(-1, 1), 1)
    y_test_onehot = y_test_onehot.to(device)

    # Define the model
    fc_model = CustomFulledConnectedModel(X_train)
    fc_model = fc_model.to(device)
    rnn_model = RNNModel(input_size=1)
    rnn_model = rnn_model.to(device)

    # Define loss, optimizer and data loader
    criterion = nn.CrossEntropyLoss()
    rnn_optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)
    fc_optimizer = optim.Adam(fc_model.parameters(), lr=0.001)
    num_epochs = 200
    batch_size = 7
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train_onehot)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Train Fullly-connected model
    for epoch in range(num_epochs):
        torch.cuda.empty_cache()

        fc_model.train()

        if epoch % 20 == 0:
          print(f'Model: Fully-Connected, Epoch: {epoch}/{num_epochs}')

        for inputs, labels in train_loader:
            fc_optimizer.zero_grad()
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = fc_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            fc_optimizer.step()

    print(f'Model: Fully-Connected, Epoch: {num_epochs}/{num_epochs}')

    fc_model.eval()

    with torch.no_grad():
        outputs = fc_model(X_test)
        predicted_labels = torch.argmax(outputs, dim=1).cpu().numpy()
        true_labels = y_test.cpu().numpy()

        mse = mean_squared_error(true_labels, predicted_labels)
        mae = mean_absolute_error(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels, average='weighted')
        score = accuracy_score(true_labels, predicted_labels)

        model_name = names[-2]
        evalutions.loc[model_name, :] = [score, f1, mse ,mae]

    torch.save(fc_model, 'full_connected.pth')


    return evalutions

In [ ]:
results = training("/content/drive/MyDrive/Composer-Classification/CorpusNoteAsChar2.txt", 'm', 13000, 5000, 5, True, True)

Running Model LogisticRegression


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Running Model PassiveAggressiveClassifier
Running Model Perceptron
Running Model RidgeClassifier
Running Model SGD Classifier
Running Model DecisionTree
Running Model BaggingClassifier
Running Model GradientBoosting
Running Model RandomForest
Running Model SVC
Running Model KNN


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model: Fully-Connected, Epoch: 0/200
Model: Fully-Connected, Epoch: 20/200
Model: Fully-Connected, Epoch: 40/200
Model: Fully-Connected, Epoch: 60/200
Model: Fully-Connected, Epoch: 80/200
Model: Fully-Connected, Epoch: 100/200
Model: Fully-Connected, Epoch: 120/200
Model: Fully-Connected, Epoch: 140/200
Model: Fully-Connected, Epoch: 160/200
Model: Fully-Connected, Epoch: 180/200
Model: Fully-Connected, Epoch: 200/200


In [ ]:
results

,score,f1-score,MSE,MAE
LogisticRegression,0.939252,0.940112,0.471963,0.107477
PassiveAggressiveClassifier,0.939252,0.934788,0.074766,0.065421
Perceptron,0.901869,0.903934,0.523364,0.186916
RidgeClassifier,0.962617,0.960359,0.107477,0.051402
SGD Classifier,0.953271,0.952891,0.046729,0.046729
DecisionTree,0.976636,0.977011,0.023364,0.023364
BaggingClassifier,0.976636,0.975883,0.023364,0.023364
GradientBoosting,0.976636,0.976359,0.098131,0.042056
RandomForest,0.990654,0.990427,0.009346,0.009346
SVC,0.953271,0.953219,0.060748,0.051402
